### Copyright 2022 Edward Späth, Frankfurt University of Applied Sciences, FB2, Computer Science
### No liability or warranty; only for educational and non-commercial purposes
### See some basic hints for working with Jupyter notebooks in README.md

## Tree traversal algorithms (Pre-, In-, Postorder) without visualization

## Global variables for storing information

In [1]:
preorder_list = []
inorder_list = []
postorder_list = []
max_index = None

## Helper functions for easier access to children/parents of a given element in tree (heap)

In [2]:
def parent(i):
    if i <= 0: # In case parent(0) is called, instead of returning -1, it is safer to return None instead, since -1 is a valid index in python
        return None
    else:
        return (i-1)//2 # '//' means division with floor function (round down)

def left(i):
    return 2*i+1

def right(i):
    return 2*i+2

## Tree traversal algorithms

In [3]:
def preorder_treewalk(x, tree):
    global preorder_list
    # Non-existing nodes are marked as 'x', which can be detected by checking whether it is a string
    if not isinstance(tree[x], str):
        # Inspect node
        preorder_list.append(tree[x])
        # Visit left
        if left(x) <= max_index:
            preorder_treewalk(left(x), tree)
        # Visit right
        if right(x) <= max_index:
            preorder_treewalk(right(x), tree)

In [4]:
def inorder_treewalk(x, tree):
    global inorder_list
    if not isinstance(tree[x], str):
        # Visit left
        if left(x) <= max_index:
            inorder_treewalk(left(x), tree)
        # Inspect node
        inorder_list.append(tree[x])
        # Visit right
        if right(x) <= max_index:
            inorder_treewalk(right(x), tree)

In [5]:
def postorder_treewalk(x, tree):
    global postorder_list
    if not isinstance(tree[x], str):
        # Visit left
        if left(x) <= max_index:
            postorder_treewalk(left(x), tree)
        # Visit right
        if right(x) <= max_index:
            postorder_treewalk(right(x), tree)
        # Inspect node
        postorder_list.append(tree[x])

## Functions for turning tree into a list which can be worked with and function to check if tree is valid

In [6]:
def turn_into_array(tree):
    # DIGITS contains Digits 0 through 9 as a list of strings
    DIGITS = []
    for j in range(0, 10):
        DIGITS.append(str(j))
    # Characters which are interpreted as a node not existing
    EMPTY_SPACES = ['x', 'X']
    tree_array = []
    index_blacklist = []
    for i in range(0, len(tree)):
        # If the current index is in the index blacklist, then it is just skipped.
        # It is used to skip over parts of a number instead of treating it as its own number.
        # Or else the number '123' would result in numbers '123', '23' and '3'.
        # This is prevented by blacklisting the index of the digit 2 and 3.
        if i in index_blacklist:
            continue
        index_blacklist = []
        # If the current character is one of the possible empty space symbols, then mark it as such
        if tree[i] in EMPTY_SPACES:
            tree_array.append('x')
        if tree[i] in DIGITS:
            j = i
            current_number = []
            # Iterate until the string of digits is interrupted (end of number)
            while tree[j] in DIGITS:
                current_number.append(tree[j])
                j += 1
            # Add indices to index blacklist for reasons stated above
            index_blacklist = list(range(i+1, i + len(current_number)))
            actual_value = 0
            # current_number contains each digit of the current number in the right order.
            # If the number were 123, then current_number would be [1, 2, 3]
            # Now by multiplying first digit with 10 to the power of 2, second with 10 and third with 1 and summing it up, we get the result.
            for index, digit in enumerate(current_number):
                actual_value += int(digit) * (10 ** (len(current_number) - index - 1))
            tree_array.append(actual_value)
    # Call a function to check whether the tree is valid (so that every node except for root has a parent)
    return check_tree(tree_array)

In [7]:
def check_tree(tree):
    # Make sure that every node has a parent (except for the root node of course)
    for i in reversed(range(1, len(tree))):
        if tree[i] != 'x' and tree[parent(i)] == 'x':
            print("\nERROR: Index", i, "is supposed to have a parent at index", parent(i), ", which does not exist")
            return None
    return tree

## Functions with does all the function calls and resets data afterwards

In [8]:
def execute_algorithms(tree):
    global max_index, preorder_list, inorder_list, postorder_list
    tree = turn_into_array(tree)
    if tree == None:
        return
    # Calculate maximum index
    max_index = len(tree)-1
    # Call algorithms
    preorder_treewalk(0, tree)
    inorder_treewalk(0, tree)
    postorder_treewalk(0, tree)
    # Print results
    print("Preorder:\n\t", preorder_list)
    print("\nInorder:\n\t", inorder_list)
    print("\nPostorder:\n\t", postorder_list)
    # Reset data
    max_index = None
    preorder_list = []
    inorder_list = []
    postorder_list = []

## Example

In [9]:
# If you want a visualized version, see "TreeTraversal.ipynb" in the same GitHub folder

# To input a binary tree, the user has to input numbers for nodes and mark where nodes are not present.
# The simplest way is by using a binary heap representation.
# Essentially imagine it as a list with an index starting at 0, increasing from top left, to bottom right (level-by-level). See below!
# Note that the dashed lines (---) and slashes/backslashes (\, /) are optional and only serve to make the tree structure more intuitive.
# The lowercase x or uppercase X marks a node not being present. They are NOT optional unless you want a full binary tree!
# After having input the last number in the lowest level everything to the right and below it can be left out. There is no need to put x's there

# tree indices:
#
#              0
#          ---   ---
#         /         \
#        1           2
#
#      /    \      /    \
#     3     4     5      6
#
#   /  \
#  7   ...

tree = """

                         48
                -------     -------
               /                   \
              18                     6

           /     \                /     \
          27     23              33     41

       /    \   /   \          /   \    /
      16    3  21    x        x     x  46


"""
execute_algorithms(tree)

Preorder:
	 [48, 18, 27, 16, 3, 23, 21, 6, 33, 41, 46]

Inorder:
	 [16, 27, 3, 18, 21, 23, 48, 33, 6, 46, 41]

Postorder:
	 [16, 3, 27, 21, 23, 18, 33, 46, 41, 6, 48]


## Yet another example

In [10]:
tree = """

              12

         /         \
        31          26

     /    \       /   \
    3      x     18    x

  /   \
 9     14


"""
execute_algorithms(tree)

Preorder:
	 [12, 31, 3, 9, 14, 26, 18]

Inorder:
	 [9, 3, 14, 31, 12, 18, 26]

Postorder:
	 [9, 14, 3, 31, 18, 26, 12]


## Your tests go here...

In [11]:
tree = """

              0

         /         \
        1           2

     /     \      /   \
    x       4    x     6

  /  \    /
 x    x  9


"""
execute_algorithms(tree)

Preorder:
	 [0, 1, 4, 9, 2, 6]

Inorder:
	 [1, 9, 4, 0, 2, 6]

Postorder:
	 [9, 4, 1, 6, 2, 0]
